# Workspace Setup

## Importing Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import container
from matplotlib.lines import Line2D
from matplotlib.collections import PolyCollection
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
%matplotlib qt
from scipy.stats import norm, lognorm
import copy
from astropy.io import fits
from astropy.table import Table


## Importing and Preprocessing Data

In [2]:
full_data = pd.read_csv('../../Data/BHcompilation_updated.csv', encoding='ISO-8859-1')

In [3]:
# Data for fitting
data = full_data.loc[full_data.SELECTED == 1]

# Normalizing uncertainties to 3-sigma values
data.loc[data.CONFLEVEL == 1, 'DMBH'] = data.DMBH*3
data.loc[data.CONFLEVEL == 2, 'DMBH'] = data.DMBH*(3/2)

# Reset the indices to avoid KeyError's later on
data = data.reset_index(drop=True)

In [4]:
# Data omitted from fits
omitted_upplim = full_data.loc[(full_data.SELECTED != 1) & (full_data.UPPERLIMIT == 1)]
omitted_true = full_data.loc[(full_data.SELECTED != 1) & (full_data.UPPERLIMIT == 0)]

# Fit data
stars_upplim = full_data.loc[(full_data.SELECTED == 1) & (full_data.MBH > 0) & (full_data.UPPERLIMIT == 1) & ((full_data.TYPE == 'star') | (full_data.TYPE == 'stars'))]
stars_upplim_zero = full_data.loc[(full_data.SELECTED == 1) & (full_data.MBH == 0) & (full_data.UPPERLIMIT == 1) & ((full_data.TYPE == 'star') | (full_data.TYPE == 'stars'))]
stars_true = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 0) & ((full_data.TYPE == 'star') | (full_data.TYPE == 'stars'))]

gas_upplim = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 1) & ((full_data.TYPE == 'gas') | (full_data.TYPE == 'CO'))]
gas_true = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 0) & ((full_data.TYPE == 'gas') | (full_data.TYPE == 'CO'))]

reverb_upplim = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 1) & (full_data.TYPE == 'reverb')]
reverb_true = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 0) & (full_data.TYPE == 'reverb')]

maser_upplim = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 1) & (full_data.TYPE == 'maser')]
maser_true = full_data.loc[(full_data.SELECTED == 1) & (full_data.UPPERLIMIT == 0) & (full_data.TYPE == 'maser')]

In [22]:
lowmass = data.loc[data.SIG < 1.8]

In [24]:
lowmass['ï»¿NAME']

10     Henize2-10
11         IC0342
26        NGC0205
32        NGC0404
33        NGC0428
37        NGC0598
43        NGC1042
60        NGC1493
80        NGC3021
97        NGC3423
102       NGC3621
189       NGC5457
202       NGC5879
210       NGC6503
217       NGC7418
218       NGC7424
225      NSA10779
226     NSA125318
227      NSA15235
228     NSA166155
229      NSA18913
230      NSA47066
231      NSA52675
233      NSA79874
234      NSA99052
Name: ï»¿NAME, dtype: object

## Function Definitions

In [5]:
# logistic function
def logit(x, beta0, beta1):
    return (1/(1+np.exp(-(beta0 + beta1*x))))
  
# log_linear function
def linear(x, gamma0, gamma1):
    return gamma0 + gamma1*x
  
# hurdle model functional form
def hurdle(x, beta0, beta1, gamma0, gamma1):
    return logit(x, beta0, beta1)*linear(x, gamma0, gamma1)

# Exploratory Data Analysis

## Data Visualization

Quick look at the raw data.

In [6]:
fig = plt.figure(figsize=(10,8), layout='constrained')
plt.xlabel(r'$\sigma$ log(km/s)', fontsize='x-large', fontweight='bold')
plt.ylabel(r'$M_\bullet$ log(M$_\odot$)', fontsize='x-large', fontweight='bold')
plt.scatter(data.SIG, data.MBH, label='data')
plt.show()

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


## Velocity Dispersion Spread

### SDSS Database

Load in the data.

In [7]:
# https://data.sdss.org/datamodel/files/SPECTRO_REDUX/galSpecLine.html

# Open the FITS file
hdul = fits.open("../../Data/galSpecLine-dr8.fits")

# Access the BinTableHDU extension
table_hdu = hdul[1]

# Convert the BinTableHDU to a Table object
data_JHU = table_hdu.data

table = Table(data_JHU)

dispersions_balmer = np.array(table['SIGMA_BALMER'])
dispersions_forbidden = np.array(table['SIGMA_FORBIDDEN'])

hdul.close

# https://data.sdss.org/datamodel/files/SPECTRO_REDUX/galSpecInfo.html

# Open the FITS file
hdul = fits.open("../../Data/galSpecInfo-dr8.fits")

# Access the BinTableHDU extension
table_hdu = hdul[1]

# Convert the BinTableHDU to a Table object
data_JHU = table_hdu.data

table = Table(data_JHU)

redshifts = np.array(table['Z'])

hdul.close

<bound method HDUList.close of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x71edf6df7eb0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x71edf6df7fa0>]>

Compare both Balmer and Forbidden lines (for velocity distribution) raw distributions.

In [8]:
plt.hist(dispersions_balmer, alpha=0.3, color='b', label='Balmer')
plt.hist(dispersions_forbidden, alpha=0.3, color='r', label='Forbidden')
plt.xlabel('Velocity Dispersion (km/s)')
plt.ylabel('Count')
plt.legend()

We will use the velocity dispersions from the Balmer lines as that is what our database is based on. But we can see that there is little to no difference between the two.

In [9]:
# Find the maximum redshift Z in our datasample
max_Z = np.max(data.Z)
max_sigma = np.max(data.SIG)

# Keep relevant galaxies (redshift lower than max redshift and lower than max sigma)
dispersions = dispersions_balmer[(redshifts >= 0) & (redshifts <= max_Z)] # removing outliers
dispersions = np.log10(dispersions[np.where(dispersions > 0)])
dispersions = dispersions[np.where((dispersions > 0) & (dispersions <= max_sigma))] # removing outliers

# Fit a normal distribution to velocity dispersion data
mu, std = norm.fit(dispersions)

fig = plt.figure(figsize=(10,8), layout='constrained')
plt.hist(dispersions, bins=25, density=True, alpha=0.6, label='MPA-JHU')
plt.hist(data.SIG, bins=25, density=True, alpha=0.6, label='Our data')

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.xlabel('Velocity Dispersion')
plt.title("Fit results: mu = %.2f,  std = %.2f" % (mu, std))

print(f'max z = {max_Z}')
print(f'mu = {mu}')
print(f'std = {std}')
plt.legend()

plt.show()

max z = 0.1014
mu = 1.8916643857955933
std = 0.268628865480423


These $\mu$ and $\sigma$ (standard deviation) values will be used in our model to generate random velocity dispersion values for posterior predictive checks.

### Our Database

Take a look at the spread of our velocity dispersions only.

In [10]:
# Fit a normal distribution to velocity dispersion data
mu, std = norm.fit(data.SIG)

fig = plt.figure(figsize=(10,8), layout='constrained')
plt.hist(data.SIG, bins=25, density=True, alpha=0.6)

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.xlabel('Velocity Dispersion')
plt.title("Fit results: mu = %.2f,  std = %.2f" % (mu, std))

print(f'mu = {mu}')
print(f'std = {std}')

plt.show()

mu = 2.143089338385246
std = 0.2660857663714269


We see that there isn't much difference between the two. Thus, it won't change much but we will still use the previous values as they are representative of the local stellar velocity dispersion distribution.

## Preliminary Fitting

In [11]:
# Fit on precise measurements
pydf = np.loadtxt('../Exploratory_Data_analysis/obs_samples.txt', delimiter=" ", dtype=float)

sigma_plot_values = np.linspace(1, 2.8, num=100)
y = np.empty((len(pydf), 100))

for i in range(len(pydf)):
    y[i] = linear(sigma_plot_values, pydf[i][0], pydf[i][1])

stan_obs_mean = np.median(y, axis=0)
stan_obs_lower = np.quantile(y, 0.975, axis=0)
stan_obs_upper = np.quantile(y, 0.025, axis=0)

# Fit on upper limits
pydf = np.loadtxt('../Exploratory_Data_analysis/upp_samples.txt', delimiter=" ", dtype=float)

sigma_plot_values = np.linspace(1, 2.8, num=100)
y = np.empty((len(pydf), 100))

for i in range(len(pydf)):
    y[i] = linear(sigma_plot_values, pydf[i][0], pydf[i][1])

stan_upp_mean = np.median(y, axis=0)
stan_upp_lower = np.quantile(y, 0.975, axis=0)
stan_upp_upper = np.quantile(y, 0.025, axis=0)

In [12]:
# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Plot labels
fig = plt.figure(figsize=(10,8), layout='constrained')
plt.xlim(1.0, 2.8)
plt.ylim(-0.5, 11)

plt.plot(sigma_plot_values, stan_obs_mean, color='black', label='Precise Measurements')
plt.fill_between(sigma_plot_values, stan_obs_lower, stan_obs_upper, alpha=0.15, color='black')

plt.plot(sigma_plot_values, stan_upp_mean, color='black', label='Upper Limits', linestyle='dotted')
plt.fill_between(sigma_plot_values, stan_upp_lower, stan_upp_upper, alpha=0.15, color='black')

# Plotting data
plt.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
plt.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
plt.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
plt.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

plt.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

plt.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
plt.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
plt.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
plt.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

plt.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
plt.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
plt.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
plt.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

plt.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
plt.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
plt.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
plt.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)


plt.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
plt.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
plt.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

# Customizing legend to add upper limit marker and remove error bars
ax = plt.gca()

# Customizing font size for axis labels
ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

handles, labels = ax.get_legend_handles_labels()

new_handles = []

for h in handles:
    # only needed to edit the errorbar legend entries
    if isinstance(h, container.ErrorbarContainer):
        new_handles.append(h[0])
    else:
        new_handles.append(h)
        
# Customizing tick marks
ax.tick_params(reset=True)
ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.xaxis.set_major_formatter('{x:.1f}')
ax.xaxis.set_minor_locator(MultipleLocator(0.1))

ax.yaxis.set_major_locator(MultipleLocator(2))
ax.yaxis.set_major_formatter('{x:.0f}')
ax.yaxis.set_minor_locator(MultipleLocator(0.5))

# Remove tick label at beginning of x axis
xticks = ax.xaxis.get_major_ticks()
xticks[1].set_visible(False)

# upper limit marker
labels.insert(2, 'upper limit') # modify the 3 depending on how many plots
new_handles.insert(2, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))


legend = ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=True, facecolor='white')
plt.grid(True, which='both', alpha=0.25)

# Plotting and saving
plt.savefig('../../Figures/exploratory_data_analysis.pdf', format='pdf')
plt.show()

# Hurdle Fit

## Chain Convergence

Let's plot the full chains (including warmup period) for each parameter to observe the convergence.

In [13]:
pydf = np.loadtxt('./samples_full.txt', delimiter=" ", dtype=float)
n_samples = pydf.shape[0]
n_chains = 4
steps = np.arange(n_samples//n_chains)

sigma_plot_values = np.linspace(1, 2.8, num=100)
y = np.empty((len(pydf), 100))

for i in range(len(pydf)):
    y[i] = hurdle(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])

stan_mean_full = np.median(y, axis=0)
stan_lower_full = np.quantile(y, 0.975, axis=0)
stan_upper_full = np.quantile(y, 0.025, axis=0)

# Split chains
chain1 = pydf[n_samples//n_chains * 0:n_samples//n_chains * 1, :]
chain2 = pydf[n_samples//n_chains * 1:n_samples//n_chains * 2, :]
chain3 = pydf[n_samples//n_chains * 2:n_samples//n_chains * 3, :]
chain4 = pydf[n_samples//n_chains * 3:n_samples//n_chains * 4, :]

#################################
# Create subplots for first param
fig, axs = plt.subplots(4, 1, sharex=True, figsize=(8, 10))

# Plot data on each subplot
axs[0].plot(steps, chain1[:, 0], label='Chain 1')
axs[1].plot(steps, chain2[:, 0], label='Chain 2')
axs[2].plot(steps, chain3[:, 0], label='Chain 3')
axs[3].plot(steps, chain4[:, 0], label='Chain 4')

axs[0].hlines(np.mean(chain1[:, 0]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain1[:, 0]):.2f}', linestyle='dashed')
axs[1].hlines(np.mean(chain2[:, 0]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain2[:, 0]):.2f}', linestyle='dashed')
axs[2].hlines(np.mean(chain3[:, 0]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain3[:, 0]):.2f}', linestyle='dashed')
axs[3].hlines(np.mean(chain4[:, 0]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain4[:, 0]):.2f}', linestyle='dashed')

# Add labels and legends
axs[0].set_ylabel(r'$\beta_0$')
axs[1].set_ylabel(r'$\beta_0$')
axs[2].set_ylabel(r'$\beta_0$')
axs[3].set_ylabel(r'$\beta_0$')

axs[3].set_xlabel('Step')

# Add legend to the last subplot
axs[0].legend(loc='upper right')
axs[1].legend(loc='upper right')
axs[2].legend(loc='upper right')
axs[3].legend(loc='upper right')
axs[0].set_title(r'Chain convergence for $\beta_0$')
plt.show()

plt.savefig('../../Figures/param_beta0.pdf', format='pdf')

##################################
# Create subplots for second param
fig, axs = plt.subplots(4, 1, sharex=True, figsize=(8, 10))

# Plot data on each subplot
axs[0].plot(steps, chain1[:, 1], label='Chain 1')
axs[1].plot(steps, chain2[:, 1], label='Chain 2')
axs[2].plot(steps, chain3[:, 1], label='Chain 3')
axs[3].plot(steps, chain4[:, 1], label='Chain 4')

axs[0].hlines(np.mean(chain1[:, 1]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain1[:, 1]):.2f}', linestyle='dashed')
axs[1].hlines(np.mean(chain2[:, 1]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain2[:, 1]):.2f}', linestyle='dashed')
axs[2].hlines(np.mean(chain3[:, 1]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain3[:, 1]):.2f}', linestyle='dashed')
axs[3].hlines(np.mean(chain4[:, 1]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain4[:, 1]):.2f}', linestyle='dashed')

# Add labels and legends
axs[0].set_ylabel(r'$\beta_1$')
axs[1].set_ylabel(r'$\beta_1$')
axs[2].set_ylabel(r'$\beta_1$')
axs[3].set_ylabel(r'$\beta_1$')

axs[3].set_xlabel('Step')

# Add legend to the last subplot
axs[0].legend(loc='upper right')
axs[1].legend(loc='upper right')
axs[2].legend(loc='upper right')
axs[3].legend(loc='upper right')
axs[0].set_title(r'Chain convergence for $\beta_1$')
plt.show()

plt.savefig('../../Figures/param_beta1.pdf', format='pdf')

#################################
# Create subplots for third param
fig, axs = plt.subplots(4, 1, sharex=True, figsize=(8, 10))

# Plot data on each subplot
axs[0].plot(steps, chain1[:, 2], label='Chain 1')
axs[1].plot(steps, chain2[:, 2], label='Chain 2')
axs[2].plot(steps, chain3[:, 2], label='Chain 3')
axs[3].plot(steps, chain4[:, 2], label='Chain 4')

axs[0].hlines(np.mean(chain1[:, 2]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain1[:, 2]):.2f}', linestyle='dashed')
axs[1].hlines(np.mean(chain2[:, 2]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain2[:, 2]):.2f}', linestyle='dashed')
axs[2].hlines(np.mean(chain3[:, 2]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain3[:, 2]):.2f}', linestyle='dashed')
axs[3].hlines(np.mean(chain4[:, 2]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain4[:, 2]):.2f}', linestyle='dashed')

# Add labels and legends
axs[0].set_ylabel(r'$\gamma_0$')
axs[1].set_ylabel(r'$\gamma_0$')
axs[2].set_ylabel(r'$\gamma_0$')
axs[3].set_ylabel(r'$\gamma_0$')

axs[3].set_xlabel('Step')

# Add legend to the last subplot
axs[0].legend(loc='upper right')
axs[1].legend(loc='upper right')
axs[2].legend(loc='upper right')
axs[3].legend(loc='upper right')
axs[0].set_title(r'Chain convergence for $\gamma_0$')
plt.show()

plt.savefig('../../Figures/param_gamma0.pdf', format='pdf')

##################################
# Create subplots for fourth param
fig, axs = plt.subplots(4, 1, sharex=True, figsize=(8, 10))

# Plot data on each subplot
axs[0].plot(steps, chain1[:, 3], label='Chain 1')
axs[1].plot(steps, chain2[:, 3], label='Chain 2')
axs[2].plot(steps, chain3[:, 3], label='Chain 3')
axs[3].plot(steps, chain4[:, 3], label='Chain 4')

axs[0].hlines(np.mean(chain1[:, 3]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain1[:, 3]):.2f}', linestyle='dashed')
axs[1].hlines(np.mean(chain2[:, 3]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain2[:, 3]):.2f}', linestyle='dashed')
axs[2].hlines(np.mean(chain3[:, 3]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain3[:, 3]):.2f}', linestyle='dashed')
axs[3].hlines(np.mean(chain4[:, 3]), steps[0], steps[-1], color='red', label=f'Mean = {np.mean(chain4[:, 3]):.2f}', linestyle='dashed')

# Add labels and legends
axs[0].set_ylabel(r'$\gamma_1$')
axs[1].set_ylabel(r'$\gamma_1$')
axs[2].set_ylabel(r'$\gamma_1$')
axs[3].set_ylabel(r'$\gamma_1$')

axs[3].set_xlabel('Step')

# Add legend to the last subplot
axs[0].legend(loc='upper right')
axs[1].legend(loc='upper right')
axs[2].legend(loc='upper right')
axs[3].legend(loc='upper right')
axs[0].set_title(r'Chain convergence for $\gamma_1$')
plt.show()

plt.savefig('../../Figures/param_gamma1.pdf', format='pdf')

As we can see, $\gamma_0$ and $\gamma_1$ are tighly constrained, whereas $\beta_0$ and $\beta_1$ (logistic parameters) are weakly constrained.

## Plotting Fit

Calculate mean parameter values and 95% Bayesian credible interval.

In [14]:
samples = np.loadtxt('./samples.txt', delimiter=" ", dtype=float)

# Calculate the median for each parameter
medians = np.median(samples, axis=0)

# Calculate the 95% credible interval for each parameter
credible_interval_low = np.percentile(samples, 2.5, axis=0)
credible_interval_high = np.percentile(samples, 97.5, axis=0)

beta0 = medians[0]
beta1 = medians[1]
gamma0 = medians[2]
gamma1 = medians[3]

beta0_err = max(credible_interval_high[0] - beta0, beta0 - credible_interval_low[0])
beta1_err = max(credible_interval_high[1] - beta1, beta1 - credible_interval_low[1])
gamma0_err = max(credible_interval_high[2] - gamma0, gamma0 - credible_interval_low[2])
gamma1_err = max(credible_interval_high[3] - gamma1, gamma1 - credible_interval_low[3])

errors = np.array([beta0_err, beta1_err, gamma0_err, gamma1_err])

# Print or use the results as needed
for param, median, low, high, error in zip(range(1, 5), medians, credible_interval_low, credible_interval_high, errors):
    print("Parameter %d: Median = %.1f, 95 Credible Interval = (%.1f, %.1f, Estimated error = %.1f)" % (param, median, low, high, error))

Parameter 1: Median = -4.4, 95 Credible Interval = (-8.9, 0.2, Estimated error = 4.5)
Parameter 2: Median = 4.3, 95 Credible Interval = (1.9, 6.9, Estimated error = 2.6)
Parameter 3: Median = -4.9, 95 Credible Interval = (-5.3, -4.4, Estimated error = 0.5)
Parameter 4: Median = 5.8, 95 Credible Interval = (5.6, 6.0, Estimated error = 0.2)


In [15]:
pydf = np.loadtxt('./samples.txt', delimiter=" ", dtype=float)

sigma_plot_values = np.linspace(1, 2.8, num=100)
y = np.empty((len(pydf), 100))
lin = np.empty((len(pydf), 100))

for i in range(len(pydf)):
    y[i] = hurdle(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])
    lin[i] = linear(sigma_plot_values, pydf[i][2], pydf[i][3])

stan_mean = np.median(y, axis=0)
stan_lower = np.quantile(y, 0.975, axis=0)
stan_upper = np.quantile(y, 0.025, axis=0)
lin_mean = np.median(lin, axis=0)

In [16]:
samples_ulimit = np.loadtxt('./samples_ulimit.txt', delimiter=" ", dtype=float)

# Calculate the median for each parameter
medians_ulimit = np.median(samples_ulimit, axis=0)

# Calculate the 95% credible interval for each parameter
credible_interval_low_ulimit = np.percentile(samples_ulimit, 2.5, axis=0)
credible_interval_high_ulimit = np.percentile(samples_ulimit, 97.5, axis=0)

beta0_ulimit = medians_ulimit[0]
beta1_ulimit = medians_ulimit[1]
gamma0_ulimit = medians_ulimit[2]
gamma1_ulimit = medians_ulimit[3]

beta0_err_ulimit = max(credible_interval_high_ulimit[0] - beta0_ulimit, beta0_ulimit - credible_interval_low_ulimit[0])
beta1_err_ulimit = max(credible_interval_high_ulimit[1] - beta1_ulimit, beta1_ulimit - credible_interval_low_ulimit[1])
gamma0_err_ulimit = max(credible_interval_high_ulimit[2] - gamma0_ulimit, gamma0_ulimit - credible_interval_low_ulimit[2])
gamma1_err_ulimit = max(credible_interval_high_ulimit[3] - gamma1_ulimit, gamma1_ulimit - credible_interval_low_ulimit[3])

errors_ulimit = np.array([beta0_err_ulimit, beta1_err_ulimit, gamma0_err_ulimit, gamma1_err_ulimit])

# Print or use the results as needed
for param, median, low, high, error in zip(range(1, 5), medians_ulimit, credible_interval_low_ulimit, credible_interval_high_ulimit, errors_ulimit):
    print("Parameter %d: Median = %.1f, 95 Credible Interval = (%.1f, %.1f, Estimated error = %.1f)" % (param, median, low, high, error))

Parameter 1: Median = -4.4, 95 Credible Interval = (-8.9, 0.2, Estimated error = 4.5)
Parameter 2: Median = 4.3, 95 Credible Interval = (1.9, 6.9, Estimated error = 2.6)
Parameter 3: Median = 3.3, 95 Credible Interval = (1.8, 5.0, Estimated error = 1.7)
Parameter 4: Median = 2.1, 95 Credible Interval = (1.2, 3.0, Estimated error = 0.9)


In [17]:
pydf_ulimit = np.loadtxt('./samples_ulimit.txt', delimiter=" ", dtype=float)
pydf_lambda = np.loadtxt('./lambda.txt', delimiter=" ", dtype=float)

sigma_plot_values_ulimit = np.linspace(1, 2.8, num=100)
y_ulimit = np.empty((len(pydf_ulimit), 100))
lin_ulimit = np.empty((len(pydf_ulimit), 100))
y_comb = np.empty((len(pydf_ulimit), 100))
lin_comb = np.empty((len(pydf_ulimit), 100))

for i in range(len(pydf_ulimit)):
    y_comb[i] = pydf_lambda[i] * hurdle(sigma_plot_values_ulimit, pydf_ulimit[i][0], pydf_ulimit[i][1], pydf_ulimit[i][2], pydf_ulimit[i][3]) + (1 - pydf_lambda[i]) * hurdle(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])
    y_ulimit[i] = hurdle(sigma_plot_values_ulimit, pydf_ulimit[i][0], pydf_ulimit[i][1], pydf_ulimit[i][2], pydf_ulimit[i][3]) + (1 - pydf_lambda[i])
    lin_ulimit[i] = linear(sigma_plot_values_ulimit, pydf_ulimit[i][2], pydf_ulimit[i][3])
    lin_comb[i] = pydf_lambda[i] * linear(sigma_plot_values_ulimit, pydf_ulimit[i][2], pydf_ulimit[i][3]) + (1 - pydf_lambda[i]) * linear(sigma_plot_values, pydf[i][2], pydf[i][3])

stan_mean_ulimit = np.median(y_ulimit, axis=0)
stan_lower_ulimit = np.quantile(y_ulimit, 0.975, axis=0)
stan_upper_ulimit = np.quantile(y_ulimit, 0.025, axis=0)
lin_mean_ulimit = np.median(lin_ulimit, axis=0)

stan_mean_comb = np.median(y_comb, axis=0)
stan_lower_comb = np.quantile(y_comb, 0.975, axis=0)
stan_upper_comb = np.quantile(y_comb, 0.025, axis=0)
lin_mean_comb = np.median(lin_comb, axis=0)

In [18]:
# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Calculating BH masses for VDB 2016
vdb_gamma0 = -4
vdb_gamma1 = 5.35
vdb_mass = linear(sigma_plot_values, vdb_gamma0, vdb_gamma1) # coefficients from Remco Van Den Bosch 2016

# Plot labels
fig = plt.figure(figsize=(10,8), layout='constrained')
plt.xlim(1.0, 2.8)
plt.ylim(-0.5, 11)

# Plotting average fit with previous ones
plt.plot(sigma_plot_values, stan_mean, color='black', label='Hurdle Model') # hurdle fit
plt.plot(sigma_plot_values, stan_mean_ulimit)
plt.fill_between(sigma_plot_values, stan_lower_ulimit, stan_upper_ulimit, alpha=0.15, color='blue') # CI
# plt.plot(sigma_plot_values, lin_mean, color='black', linestyle='dashed', label='Linear Portion') # linear portion
plt.plot(sigma_plot_values, vdb_mass, color='black', linestyle='dotted', label='van den Bosch 2016')
plt.fill_between(sigma_plot_values, stan_lower, stan_upper, alpha=0.15, color='black', label='95% Credible Interval') # CI

# Plotting data
plt.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
plt.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
plt.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
plt.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

plt.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

plt.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
plt.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
plt.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
plt.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

plt.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
plt.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
plt.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
plt.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

plt.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
plt.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
plt.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
plt.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)


plt.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
plt.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
plt.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

# Customizing legend to add upper limit marker and remove error bars
ax = plt.gca()

# ax.text(1.64, 1.70, '$\\log M_\\bullet = p(%.1f^{\\pm%.1f}+%.1f^{\\pm%.1f}\\log\\sigma)$' % (gamma0, gamma0_err, gamma1, gamma1_err), fontsize=22)
# ax.text(1.66, 1.70, '$\\log M$', fontsize=22)
# ax.text(1.80, 1.55, '$\\bullet$', fontsize=32)
# ax.text(1.86, 1.70, '$= p(%.1f^{\\pm%.1f}+%.1f^{\\pm%.1f}\\log\\sigma)$' % (gamma0, gamma0_err, gamma1, gamma1_err), fontsize=22)
# ax.text(1.82, 0.70, '$p = \\frac{1}{1+exp{(-(%.1f^{\\pm%.1f}+%.1f^{\\pm%.1f}\\log\\sigma))}}$' % (beta0, beta0_err, beta1, beta1_err), fontsize=22)

# Customizing font size for axis labels
ax.text(0.88, 3.85, r'BH mass $M$', ha='center', va='center', rotation=90, fontsize=25)
ax.text(0.9, 5.45, r'$\bullet$', ha='center', va='center', rotation=90, fontsize=35)
ax.text(0.88, 7.15, r'(log $M_\odot$)', ha='center', va='center', rotation=90, fontsize=25)
ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
# ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

handles, labels = ax.get_legend_handles_labels()

new_handles = []

for h in handles:
    # only needed to edit the errorbar legend entries
    if isinstance(h, container.ErrorbarContainer):
        new_handles.append(h[0])
    else:
        new_handles.append(h)
        
# Customizing tick marks
ax.tick_params(reset=True)
ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.xaxis.set_major_formatter('{x:.1f}')
ax.xaxis.set_minor_locator(MultipleLocator(0.1))

ax.yaxis.set_major_locator(MultipleLocator(2))
ax.yaxis.set_major_formatter('{x:.0f}')
ax.yaxis.set_minor_locator(MultipleLocator(0.5))

# Remove tick label at beginning of x axis
xticks = ax.xaxis.get_major_ticks()
xticks[1].set_visible(False)

# upper limit marker
labels.insert(4, 'upper limit') # modify the 3 depending on how many plots
new_handles.insert(4, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))


legend = ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=True, facecolor='white')
plt.grid(True, which='both', alpha=0.25)

# Plotting and saving
# plt.savefig('../../Figures/Levy_Upper_Lims.pdf', format='pdf')
plt.savefig('/home/gsasseville/Downloads/pres2.png')
plt.show()

In [19]:
# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Calculating BH masses for VDB 2016
vdb_gamma0 = -4
vdb_gamma1 = 5.35
vdb_mass = linear(sigma_plot_values, vdb_gamma0, vdb_gamma1) # coefficients from Remco Van Den Bosch 2016

# Plot labels
fig = plt.figure(figsize=(10,8), layout='constrained')
plt.xlim(1.0, 2.8)
plt.ylim(-0.5, 11)

# Plotting average fit with previous ones
plt.plot(sigma_plot_values, stan_mean_comb, color='black', label='Hurdle Model') # hurdle fit
plt.plot(sigma_plot_values, lin_mean_comb, color='black', linestyle='dashed', label='Linear Portion') # linear portion
plt.plot(sigma_plot_values, vdb_mass, color='black', linestyle='dotted', label='van den Bosch 2016')
plt.fill_between(sigma_plot_values, stan_lower_comb, stan_upper_comb, alpha=0.15, color='black', label='95% Credible Interval') # CI

# Plotting data
plt.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
plt.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
plt.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
plt.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

plt.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

plt.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
plt.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
plt.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
plt.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

plt.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
plt.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
plt.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
plt.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

plt.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
plt.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
plt.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
plt.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)


plt.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
plt.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
plt.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

# Customizing legend to add upper limit marker and remove error bars
ax = plt.gca()

# ax.text(1.64, 1.70, '$\\log M_\\bullet = p(%.1f^{\\pm%.1f}+%.1f^{\\pm%.1f}\\log\\sigma)$' % (gamma0, gamma0_err, gamma1, gamma1_err), fontsize=22)
# ax.text(1.66, 1.70, '$\\log M$', fontsize=22)
# ax.text(1.80, 1.55, '$\\bullet$', fontsize=32)
# ax.text(1.86, 1.70, '$= p(%.1f^{\\pm%.1f}+%.1f^{\\pm%.1f}\\log\\sigma)$' % (gamma0, gamma0_err, gamma1, gamma1_err), fontsize=22)
# ax.text(1.82, 0.70, '$p = \\frac{1}{1+exp{(-(%.1f^{\\pm%.1f}+%.1f^{\\pm%.1f}\\log\\sigma))}}$' % (beta0, beta0_err, beta1, beta1_err), fontsize=22)

# Customizing font size for axis labels
ax.text(0.88, 3.85, r'BH mass $M$', ha='center', va='center', rotation=90, fontsize=25)
ax.text(0.9, 5.45, r'$\bullet$', ha='center', va='center', rotation=90, fontsize=35)
ax.text(0.88, 7.15, r'(log $M_\odot$)', ha='center', va='center', rotation=90, fontsize=25)
ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
# ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

handles, labels = ax.get_legend_handles_labels()

new_handles = []

for h in handles:
    # only needed to edit the errorbar legend entries
    if isinstance(h, container.ErrorbarContainer):
        new_handles.append(h[0])
    else:
        new_handles.append(h)
        
# Customizing tick marks
ax.tick_params(reset=True)
ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.xaxis.set_major_formatter('{x:.1f}')
ax.xaxis.set_minor_locator(MultipleLocator(0.1))

ax.yaxis.set_major_locator(MultipleLocator(2))
ax.yaxis.set_major_formatter('{x:.0f}')
ax.yaxis.set_minor_locator(MultipleLocator(0.5))

# Remove tick label at beginning of x axis
xticks = ax.xaxis.get_major_ticks()
xticks[1].set_visible(False)

# upper limit marker
labels.insert(4, 'upper limit') # modify the 3 depending on how many plots
new_handles.insert(4, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))


legend = ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=True, facecolor='white')
plt.grid(True, which='both', alpha=0.25)

# Plotting and saving
# plt.savefig('../../Figures/Levy_Upper_Lims.pdf', format='pdf')
plt.savefig('/home/gsasseville/Downloads/pres2.png')
plt.show()

## Analysis

### Posterior Distributions

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Assuming 'linear' function and other necessary code is already defined

# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Calculating BH masses for VDB 2016
vdb_gamma0 = -4
vdb_gamma1 = 5.35
vdb_mass = linear(sigma_plot_values, vdb_gamma0, vdb_gamma1)  # coefficients from Remco Van Den Bosch 2016

# Creating figure and gridspec layout
fig = plt.figure(figsize=(12, 10))
gs = GridSpec(2, 2, width_ratios=[3, 1], height_ratios=[1, 3], wspace=0.05, hspace=0.05)

# Main plot
ax_main = fig.add_subplot(gs[1, 0])

# Plotting fits
ax_main.plot(sigma_plot_values, stan_mean, color='black', label='Hurdle Model')  # hurdle fit
ax_main.plot(sigma_plot_values, stan_mean_ulimit)
ax_main.fill_between(sigma_plot_values, stan_lower_ulimit, stan_upper_ulimit, alpha=0.15, color='blue')  # CI
ax_main.plot(sigma_plot_values, lin_mean, color='black', linestyle='dashed', label='Linear Portion')  # linear portion
ax_main.plot(sigma_plot_values, vdb_mass, color='black', linestyle='dotted', label='van den Bosch 2016')
ax_main.fill_between(sigma_plot_values, stan_lower, stan_upper, alpha=0.15, color='black', label='95% Credible Interval')  # CI

# Plotting data points
ax_main.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
ax_main.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
ax_main.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
ax_main.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

ax_main.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
ax_main.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
ax_main.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
ax_main.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

ax_main.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
ax_main.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
ax_main.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
ax_main.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

ax_main.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
ax_main.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
ax_main.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
ax_main.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)

ax_main.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
ax_main.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
ax_main.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

ax_main.legend()
ax_main.set_xlabel('Velocity Dispersion (log km/s)')
ax_main.set_ylabel('BH Mass (log $M_\odot$)')
ax_main.set_xlim(1.0, 2.8)
ax_main.set_ylim(-0.5, 11)

# Posterior distribution histograms
ax_hist = fig.add_subplot(gs[1, 1], sharey=ax_main)

fixed_sigma = 1.5

ax_main.vlines(fixed_sigma, -0.5, 11, color='red', linestyles='dotted')

posterior_samples = np.loadtxt('./samples.txt', delimiter=" ", dtype=float)
hist_data = np.array([linear(fixed_sigma, row[2], row[3]) for row in posterior_samples])

posterior_samples_ulimit = np.loadtxt('./samples_ulimit.txt', delimiter=" ", dtype=float)
hist_data_ulimit = np.array([linear(fixed_sigma, row[2], row[3]) for row in posterior_samples_ulimit])

ax_hist.hist(hist_data, bins=30, orientation='horizontal', color='orange', alpha=0.7, label='Posterior')
ax_hist.hist(hist_data_ulimit, bins=30, orientation='horizontal', color='blue', alpha=0.7, label='Posterior Upper Limit')
ax_hist.set_xlabel('Density')
ax_hist.set_yticks([])

ax_hist.legend()

plt.savefig('/home/gsasseville/Downloads/pres2.png')
plt.show()


### Comparison Upper Limits VS No Upper Limits

In [21]:
pydf = np.loadtxt('../LogNorm_Uncertainties_No_Upper_Limits/samples.txt', delimiter=" ", dtype=float)

sigma_plot_values = np.linspace(1, 2.8, num=100)
y = np.empty((len(pydf), 100))
lin = np.empty((len(pydf), 100))

for i in range(len(pydf)):
    y[i] = hurdle(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])
    lin[i] = linear(sigma_plot_values, pydf[i][2], pydf[i][3])

stan_mean = np.median(y, axis=0)
stan_lower = np.quantile(y, 0.975, axis=0)
stan_upper = np.quantile(y, 0.025, axis=0)

lin_mean = np.median(lin, axis=0)

np.mean(pydf, axis=0)

pydf = np.loadtxt('./samples.txt', delimiter=" ", dtype=float)

sigma_plot_values = np.linspace(1, 2.8, num=100)
y = np.empty((len(pydf), 100))

for i in range(len(pydf)):
    y[i] = hurdle(sigma_plot_values, pydf[i][0], pydf[i][1], pydf[i][2], pydf[i][3])

stan_levy_mean = np.median(y, axis=0)
stan_levy_lower = np.quantile(y, 0.975, axis=0)
stan_levy_upper = np.quantile(y, 0.025, axis=0)

In [22]:
# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Calculating fit with confidence interval

# Plot labels
fig = plt.figure(figsize=(10,8), layout='constrained')
plt.xlim(1.0, 2.8)
plt.ylim(-0.5, 11)

# Plotting average fit with previous ones

plt.plot(sigma_plot_values, stan_levy_mean, color='black', label='Lévy Upper Limits')
plt.fill_between(sigma_plot_values, stan_levy_lower, stan_levy_upper, alpha=0.15, color='black')

plt.plot(sigma_plot_values, stan_mean, color='green', label='Lognormal Upper Limits')
plt.fill_between(sigma_plot_values, stan_lower, stan_upper, alpha=0.15, color='green')

# Plotting data
plt.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
plt.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
plt.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
plt.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

plt.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

plt.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
plt.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
plt.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
plt.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

plt.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
plt.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
plt.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
plt.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

plt.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
plt.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
plt.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
plt.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)


plt.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
plt.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
plt.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

# Customizing legend to add upper limit marker and remove error bars
ax = plt.gca()

# Customizing font size for axis labels
ax.text(0.88, 3.85, r'BH mass $M$', ha='center', va='center', rotation=90, fontsize=25)
ax.text(0.9, 5.45, r'$\bullet$', ha='center', va='center', rotation=90, fontsize=35)
ax.text(0.88, 7.15, r'(log $M_\odot$)', ha='center', va='center', rotation=90, fontsize=25)
ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
# ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

handles, labels = ax.get_legend_handles_labels()

new_handles = []

for h in handles:
    # only needed to edit the errorbar legend entries
    if isinstance(h, container.ErrorbarContainer):
        new_handles.append(h[0])
    else:
        new_handles.append(h)
        
# Customizing tick marks
ax.tick_params(reset=True)
ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.xaxis.set_major_formatter('{x:.1f}')
ax.xaxis.set_minor_locator(MultipleLocator(0.1))

ax.yaxis.set_major_locator(MultipleLocator(2))
ax.yaxis.set_major_formatter('{x:.0f}')
ax.yaxis.set_minor_locator(MultipleLocator(0.5))

# Remove tick label at beginning of x axis
xticks = ax.xaxis.get_major_ticks()
xticks[1].set_visible(False)

# upper limit marker
labels.insert(2, 'upper limit') # modify the 3 depending on how many plots
new_handles.insert(2, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))

legend = ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=True, facecolor='white')
plt.grid(True, which='both', alpha=0.25)

# Plotting and saving
plt.savefig('../../Figures/upperlimits_vs_noupperlimits.pdf', format='pdf')
plt.show()

### Logistic Portion

Calculate the stellar velocity dispersion values at which the 50%, 90% and 99% probability of hosting a central BH is attained. Calculate what percentage of the data set is above these values of velocity dispersions.

In [36]:
# Probability values
P = [0.5, 0.9, 0.99]
sigma_vals = np.zeros(len(P))
mass_vals = np.zeros(len(P))
percentages = np.zeros(len(P))

# Calculate velocity dispersions and masses corresponding to P vals
for p, i in zip(P, range(len(P))):
    sigma_vals[i] = (1/beta1 * (np.log(p/(1-p)) - beta0))
    mass_vals[i] = (gamma0 + gamma1*sigma_vals[i])
    print('For p = %.2f, we have velocity dispersion = %.0f km/s and BH mass = %.1E solar masses' % (p, 10**sigma_vals[i], 10**mass_vals[i]))

print('')

# Calculate percentage of data
for i in range(len(P)):
    print('%.2f%% of the galaxies are above the p = %.2f probability of having a BH' % (len(np.where(data.SIG > sigma_vals[i])[0])/len(data.SIG), P[i]))

For p = 0.50, we have velocity dispersion = 11 km/s and BH mass = 1.7E+02 solar masses
For p = 0.90, we have velocity dispersion = 35 km/s and BH mass = 5.1E+04 solar masses
For p = 0.99, we have velocity dispersion = 125 km/s and BH mass = 2.6E+07 solar masses

1.00% of the galaxies are above the p = 0.50 probability of having a BH
0.96% of the galaxies are above the p = 0.90 probability of having a BH
0.63% of the galaxies are above the p = 0.99 probability of having a BH


Calculate probability of hosting a BH at $M_\bullet = 10^4 M_\odot$.

In [37]:
sigma_for_bh_mass_4 = (4 - gamma0)/gamma1
probability_for_bh_mass_4 = logit(sigma_for_bh_mass_4, beta0, beta1)*100

print('There is a %.0f%% chance that a galaxy with velocity dispersion of %.0f km/s (corresponding to BH mass of 10^4 solar masses) contains a central BH given our hurdle model.' % (probability_for_bh_mass_4, 10**sigma_for_bh_mass_4))

There is a 83% chance that a galaxy with velocity dispersion of 25 km/s (corresponding to BH mass of 10^4 solar masses) contains a central BH given our hurdle model.


### Linear Portion

Calculate the velocity dispersion and BH mass values at which our linear curve intersects VDB2016.

In [38]:
transition_sigma = (vdb_gamma0 - gamma0)/(gamma1 - vdb_gamma1)
transition_mass = gamma0 + gamma1*transition_sigma

print('Transition velocity dispersion is: %.0f' % 10**transition_sigma)
print('Transition mass is: %.1E' % 10**transition_mass)

Transition velocity dispersion is: 292
Transition mass is: 1.5E+09


Calculate the velocity dispersion at which BHs are ultra-massive, i.e. masses are greater than $10^{10}$

In [39]:
sigma_ultramassive = (10 - gamma0)/gamma1

print('BHs have masses greater than 10^10 at velocity dispersion: %.0f' % 10**sigma_ultramassive)

BHs have masses greater than 10^10 at velocity dispersion: 430


# Predictive Checks

## Prior Predictive Checks

Verify weak priors.

In [40]:
# Load data
pydf = np.loadtxt('./prior_samples_full.txt', delimiter=" ", dtype=float)

beta0_priors = pydf[:, 0]
beta1_priors = pydf[:, 1]
gamma0_priors = pydf[:, 2]
gamma1_priors = pydf[:, 3]

# Create a 2x2 grid of subplots
plt.figure(figsize=(10, 8))

# Top row, first subplot
plt.subplot(2, 2, 1)
hist_beta0, _, _ = plt.hist(beta0_priors, color='blue', alpha=0.7)
ymin, ymax = plt.ylim()
# plt.xlim(-1000, 1000)
plt.vlines(beta0, ymin, ymax)
plt.title('Beta0 Priors')

# Top row, second subplot
plt.subplot(2, 2, 2)
hist_beta1, _, _ = plt.hist(beta1_priors, color='orange', alpha=0.7)
ymin, ymax = plt.ylim()
# plt.xlim(-1000, 1000)
plt.vlines(beta1, ymin, ymax)
plt.title('Beta1 Priors')

# Bottom row, first subplot
plt.subplot(2, 2, 3)
hist_gamma0, _, _ = plt.hist(gamma0_priors, color='green', alpha=0.7)
ymin, ymax = plt.ylim()
# plt.xlim(-1000, 1000)
plt.vlines(gamma0, ymin, ymax)
plt.title('Gamma0 Priors')

# Bottom row, second subplot
plt.subplot(2, 2, 4)
hist_gamma1, _, _ = plt.hist(gamma1_priors, color='red', alpha=0.7)
ymin, ymax = plt.ylim()
# plt.xlim(-1000, 1000)
plt.vlines(gamma1, ymin, ymax)
plt.title('Gamma1 Priors')

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()

In [41]:
x_rep_arr = np.loadtxt('./prior_pred_check_x.txt', delimiter=" ", dtype=float)
lin_arr = np.loadtxt('./prior_pred_check_lin.txt', delimiter=" ", dtype=float)
hur_arr = np.loadtxt('./prior_pred_check_hur.txt', delimiter=" ", dtype=float)

In [42]:
color_sim = '#dbbc23'

# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Plot labels
fig = plt.figure(figsize=(10,8), layout='constrained')
plt.xlim(1.0, 2.8)

# Plotting simulated data
n_sims = 1000
sims = np.random.randint(0, len(lin_arr), size=n_sims)
x = np.array(data.SIG)
for i in sims:
    x_rep = x_rep_arr[i]
    lin = lin_arr[i]
    hur = hur_arr[i]

    upp_lim_idx = np.where(hur == 0)
    non_upp_lim_idx = np.where(hur == 1)
    plt.scatter(x_rep[upp_lim_idx], lin[upp_lim_idx], color='#dbbc23', marker='v', s=50, edgecolors='black')
    plt.scatter(x_rep[non_upp_lim_idx], lin[non_upp_lim_idx], color='#dbbc23', s=36)


# Plotting true data
plt.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
plt.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
plt.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
plt.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

plt.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

plt.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
plt.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
plt.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
plt.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

plt.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
plt.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
plt.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
plt.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

plt.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
plt.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
plt.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
plt.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)


plt.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
plt.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
plt.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

# Customizing legend to add upper limit marker and remove error bars
ax = plt.gca()

# Customizing font size for axis labels
ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

handles, labels = ax.get_legend_handles_labels()

new_handles = []

for h in handles:
    # only needed to edit the errorbar legend entries
    if isinstance(h, container.ErrorbarContainer):
        new_handles.append(h[0])
    else:
        new_handles.append(h)

# upper limit marker
labels.insert(0, 'upper limit') # modify the 3 depending on how many plots
new_handles.insert(0, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))
labels.insert(6, 'simulated') # modify the 3 depending on how many plots
new_handles.insert(6, Line2D([0], [0], color='#dbbc23', marker='.', ls=''))

legend = ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=True, facecolor='white')
plt.grid(True, which='both', alpha=0.25)

# Plotting and saving
plt.show()

## Posterior Predictive Checks

Load in data.

In [43]:
x_rep_arr = np.loadtxt('./post_pred_check_x.txt', delimiter=" ", dtype=float)
lin_arr = np.loadtxt('./post_pred_check_lin.txt', delimiter=" ", dtype=float)
hur_arr = np.loadtxt('./post_pred_check_hur.txt', delimiter=" ", dtype=float)

x_rep_arr_10x = np.loadtxt('./post_pred_check_x_10x.txt', delimiter=" ", dtype=float)
lin_arr_10x = np.loadtxt('./post_pred_check_lin_10x.txt', delimiter=" ", dtype=float)
hur_arr_10x = np.loadtxt('./post_pred_check_hur_10x.txt', delimiter=" ", dtype=float)

x_rep_arr_100x = np.loadtxt('./post_pred_check_x_100x.txt', delimiter=" ", dtype=float)
lin_arr_100x = np.loadtxt('./post_pred_check_lin_100x.txt', delimiter=" ", dtype=float)
hur_arr_100x = np.loadtxt('./post_pred_check_hur_100x.txt', delimiter=" ", dtype=float)

In [44]:
# setup simulation parameters
np.random.seed(1) # set seed for reproducibility
n_sims = 1000 # number of random posterior samples
sims = np.random.randint(0, len(lin_arr), size=n_sims)

### Analyzing predictive checks

Calculate above what value of velocity dispersion are situated the upper 75% of galaxies

In [45]:
sig_cutoff = 1.6
counter_above_cutoff = 0
counter_total = 0

for i in sims:
    x_rep = copy.deepcopy(x_rep_arr[i])
    lin = copy.deepcopy(lin_arr[i])
    hur = copy.deepcopy(hur_arr[i])

    upp_lim_idx = np.where(hur == 0)

    counter_total += len(x_rep)
    counter_above_cutoff += len(np.where(x_rep[upp_lim_idx] > sig_cutoff)[0])

percentage_observed = len(np.where(np.array(data.SIG)[np.where(data.MBH == 0)] > sig_cutoff)[0])/len(data.MBH) * 100
percentage_simulated = counter_above_cutoff/counter_total * 100

print('%.1f %% of simulated data under interpretation of the right panel correspond to zeros meanwhile it is only %.1f %% for observed data.' % (percentage_simulated, percentage_observed))

2.1 % of simulated data under interpretation of the right panel correspond to zeros meanwhile it is only 0.8 % for observed data.


In [46]:
tmp_x = np.array([])

for i in sims:
    x_rep = copy.deepcopy(x_rep_arr[i])
    lin = copy.deepcopy(lin_arr[i])
    hur = copy.deepcopy(hur_arr[i])

    upp_lim_idx = np.where(hur == 0)

    tmp_x = np.append(tmp_x, x_rep[upp_lim_idx])

sig1 = np.percentile(tmp_x, [25.4])[0]

print('75%% of the simulated upper limits fall above stellar velocity dispersion value of %.0f km/s.' % (10**sig1))
print('Using our linear portion, this corresponds to a BH mass of %.1E solar masses.' % (10**linear(sig1, gamma0, gamma1)))

75% of the simulated upper limits fall above stellar velocity dispersion value of 35 km/s.
Using our linear portion, this corresponds to a BH mass of 5.6E+04 solar masses.


Calculate what fraction of simulated data do BHs with $M_\bullet > 10^{10} M_\odot$ represent. Compare this with our observational data.

In [47]:
# Observed data
obs_BH_masses = np.array(data.MBH)
geq_10_obs_BH_masses = obs_BH_masses[np.where(obs_BH_masses > 10)]
percentage_obs_geq_10 = len(geq_10_obs_BH_masses)/len(obs_BH_masses)*100

# Simulated data
sim_BH_masses = lin_arr_100x[sims].flatten()
geq_10_sim_BH_masses = sim_BH_masses[np.where(sim_BH_masses > 10)]
percentage_sim_geq_10 = len(geq_10_sim_BH_masses)/len(sim_BH_masses)*100

print('%.1f%% of observed BHs have masses greater than 10^10, meanwhile for the simulated data it is %.1f%%' % (percentage_obs_geq_10, percentage_sim_geq_10))

0.8% of observed BHs have masses greater than 10^10, meanwhile for the simulated data it is 4.7%


### Simulating 10 Points Per Posterior Sample

In [48]:
# Getting simulated data. Putting upper limits at zero
hur_flat = copy.deepcopy(hur_arr[sims]).flatten()
ul_idx_flat = np.where(hur_flat == 0)
lin_flat = copy.deepcopy(lin_arr[sims]).flatten()
lin_flat[ul_idx_flat] = 0

bin_edges = np.linspace(0, 14, num=50)  # Adjust num as needed

# Plotting histograms with common bin edges
fig = plt.figure(figsize=(12,14))
plt.hist(data.MBH, bins=bin_edges, alpha=0.5, density=True, color='dodgerblue', label='Observed Data')
plt.hist(lin_flat, bins=bin_edges, alpha=0.5, density=True, color=color_sim, label='Simulated Data')

ax = plt.gca()
ax.set_xlim(-0.5, 13)
ax.set_ylim(0, 0.375)
# Customizing font size for axis labels
ax.text(5.35, -0.02, r'BH mass $M$', ha='center', va='top', fontsize=25)
ax.text(7.20, -0.02, r'$\bullet$', ha='center', va='top', fontsize=35)
ax.text(8.90, -0.02, r'(log $M_\odot$)', ha='center', va='top', fontsize=25)
# ax.set_xlabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)
ax.set_ylabel(r'Density', fontsize=25)

# Customizing ticks
ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_major_formatter('{x:.0f}')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))

ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_major_formatter('{x:.1f}')
ax.yaxis.set_minor_locator(MultipleLocator(0.025))

# Remove tick labels at beginning and end
yticks = ax.yaxis.get_major_ticks()
yticks[1].set_visible(False)

# Adding legend
ax.legend(loc='upper left', fontsize='xx-large', markerscale=2.5, frameon=True, facecolor='white')
ax.grid(True, which='both', alpha=0.25)

plt.savefig('../../Figures/histogram_comparison.pdf', format='pdf')

In [49]:
# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Plot labels
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8), sharey=False)

for ax, j in zip(axes, range(len(axes))):
    ax.set_xlim(1.0, 2.8)
    ax.set_ylim(-0.5, 11)

    # Plotting simulated data
    for i in sims:
        x_rep = copy.deepcopy(x_rep_arr[i])
        lin = copy.deepcopy(lin_arr[i])
        hur = copy.deepcopy(hur_arr[i])

        upp_lim_idx = np.where(hur == 0)
        non_upp_lim_idx = np.where(hur == 1)
        
        if j == 1:
            lin[upp_lim_idx] = 0
            ax.scatter(x_rep[upp_lim_idx], lin[upp_lim_idx], color=color_sim, marker='v', s=50, alpha=0.3)
        else:
            ax.scatter(x_rep[upp_lim_idx], lin[upp_lim_idx], color=color_sim, marker='v', s=50, edgecolors='black', alpha=0.3)

        ax.scatter(x_rep[non_upp_lim_idx], lin[non_upp_lim_idx], color=color_sim, s=36, alpha=0.3)

    # Plotting true data
    ax.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
    ax.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
    ax.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
    ax.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

    ax.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

    ax.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
    ax.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
    ax.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
    ax.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

    ax.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
    ax.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
    ax.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
    ax.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

    ax.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
    ax.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
    ax.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
    ax.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)

    ax.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
    ax.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
    ax.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

    # Customizing font size for axis labels
    ax.text(0.88, 3.85, r'BH mass $M$', ha='center', va='center', rotation=90, fontsize=25)
    ax.text(0.9, 5.7, r'$\bullet$', ha='center', va='center', rotation=90, fontsize=35)
    ax.text(0.88, 7.5, r'(log $M_\odot$)', ha='center', va='center', rotation=90, fontsize=25)
    ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
    # ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

    # Customizing legend to add upper limit marker and remove error bars
    handles, labels = ax.get_legend_handles_labels()

    new_handles = []

    for h in handles:
        # only needed to edit the errorbar legend entries
        if isinstance(h, container.ErrorbarContainer):
            new_handles.append(h[0])
        else:
            new_handles.append(h)

    ax.tick_params(reset=True)
    ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
    ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
    ax.xaxis.set_major_locator(MultipleLocator(0.5))
    ax.xaxis.set_major_formatter('{x:.1f}')
    ax.xaxis.set_minor_locator(MultipleLocator(0.1))

    ax.yaxis.set_major_locator(MultipleLocator(2))
    ax.yaxis.set_major_formatter('{x:.0f}')
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))

    # Remove tick label at beginning of x axis
    xticks = ax.xaxis.get_major_ticks()
    xticks[1].set_visible(False)

    # upper limit marker
    labels.insert(0, 'upper limit')  # modify the index depending on how many plots
    new_handles.insert(0, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))
    labels.insert(6, 'simulated')  # modify the index depending on how many plots
    new_handles.insert(6, Line2D([0], [0], color=color_sim, marker='.', ls=''))

    legend = ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=True, facecolor='white')
    ax.grid(True, which='both', alpha=0.25)

    # Add a) and b) labels
    if j == 0:
        ax.text(0.02, 1.05, f'a) Logistic Zeros Plotted As Upper Limits', transform=ax.transAxes, va='top', ha='left', fontsize=20)
    else:
        ax.text(0.02, 1.05, f'b) Logistic Zeros Plotted As Galaxies Without BHs', transform=ax.transAxes, va='top', ha='left', fontsize=20)

# Plotting and saving
plt.savefig('../../Figures/simulated_data.pdf', format='pdf')
plt.show()

### Simulating 100 Points Per Posterior Sample

In [50]:
# Getting simulated data. Putting upper limits at zero
hur_flat_10x = copy.deepcopy(hur_arr_10x[sims]).flatten()
ul_idx_flat_10x = np.where(hur_flat_10x == 0)
lin_flat_10x = copy.deepcopy(lin_arr_10x[sims]).flatten()
lin_flat_10x[ul_idx_flat_10x] = 0

bin_edges = np.linspace(0, 14, num=50)  # Adjust num as needed

# Plotting histograms with common bin edges
fig = plt.figure(figsize=(12,14))
plt.hist(data.MBH, bins=bin_edges, alpha=0.5, density=True, color='dodgerblue', label='Observed Data')
plt.hist(lin_flat_10x, bins=bin_edges, alpha=0.5, density=True, color=color_sim, label='Simulated Data')

ax = plt.gca()
ax.set_xlim(-0.5, 13)
ax.set_ylim(0, 0.375)
# Customizing font size for axis labels
ax.text(5.35, -0.02, r'BH mass $M$', ha='center', va='top', fontsize=25)
ax.text(7.20, -0.02, r'$\bullet$', ha='center', va='top', fontsize=35)
ax.text(8.90, -0.02, r'(log $M_\odot$)', ha='center', va='top', fontsize=25)
# ax.set_xlabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)
ax.set_ylabel(r'Density', fontsize=25)

# Customizing ticks
ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_major_formatter('{x:.0f}')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))

ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_major_formatter('{x:.1f}')
ax.yaxis.set_minor_locator(MultipleLocator(0.05))

# Remove tick label at beginning of x axis
yticks = ax.yaxis.get_major_ticks()
yticks[1].set_visible(False)

# Adding legend
ax.legend(loc='upper left', fontsize='xx-large', markerscale=2.5, frameon=True, facecolor='white')
ax.grid(True, which='both', alpha=0.25)

plt.savefig('../../Figures/histogram_comparison_10x.pdf', format='pdf')

In [51]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import container
import numpy as np

# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Plot labels
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8), sharey=False)

for ax, j in zip(axes, range(len(axes))):
    ax.set_xlim(1.0, 2.8)
    ax.set_ylim(-0.5, 11)

    # Plotting simulated data
    for i in sims:
        x_rep = copy.deepcopy(x_rep_arr_10x[i])
        lin = copy.deepcopy(lin_arr_10x[i])
        hur = copy.deepcopy(hur_arr_10x[i])

        upp_lim_idx = np.where(hur == 0)
        non_upp_lim_idx = np.where(hur == 1)
        
        if j == 1:
            lin[upp_lim_idx] = 0
            ax.scatter(x_rep[upp_lim_idx], lin[upp_lim_idx], color=color_sim, marker='v', s=50, alpha=0.3)
        else:
            ax.scatter(x_rep[upp_lim_idx], lin[upp_lim_idx], color=color_sim, marker='v', s=50, edgecolors='black', alpha=0.3)

        ax.scatter(x_rep[non_upp_lim_idx], lin[non_upp_lim_idx], color=color_sim, s=36, alpha=0.3)

    # Plotting true data
    ax.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
    ax.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
    ax.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
    ax.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

    ax.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

    ax.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
    ax.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
    ax.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
    ax.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

    ax.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
    ax.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
    ax.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
    ax.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

    ax.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
    ax.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
    ax.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
    ax.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)

    ax.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
    ax.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
    ax.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

    # Customizing font size for axis labels
    ax.text(0.88, 3.85, r'BH mass $M$', ha='center', va='center', rotation=90, fontsize=25)
    ax.text(0.9, 5.7, r'$\bullet$', ha='center', va='center', rotation=90, fontsize=35)
    ax.text(0.88, 7.5, r'(log $M_\odot$)', ha='center', va='center', rotation=90, fontsize=25)
    ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
    # ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

    # Customizing legend to add upper limit marker and remove error bars
    handles, labels = ax.get_legend_handles_labels()

    new_handles = []

    for h in handles:
        # only needed to edit the errorbar legend entries
        if isinstance(h, container.ErrorbarContainer):
            new_handles.append(h[0])
        else:
            new_handles.append(h)
            
    ax.tick_params(reset=True)
    ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
    ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
    ax.xaxis.set_major_locator(MultipleLocator(0.5))
    ax.xaxis.set_major_formatter('{x:.1f}')
    ax.xaxis.set_minor_locator(MultipleLocator(0.1))

    ax.yaxis.set_major_locator(MultipleLocator(2))
    ax.yaxis.set_major_formatter('{x:.0f}')
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))

    # Remove tick label at beginning of x axis
    xticks = ax.xaxis.get_major_ticks()
    xticks[1].set_visible(False)

    # upper limit marker
    labels.insert(0, 'upper limit')  # modify the index depending on how many plots
    new_handles.insert(0, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))
    labels.insert(6, 'simulated')  # modify the index depending on how many plots
    new_handles.insert(6, Line2D([0], [0], color=color_sim, marker='.', ls=''))

    # ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=False)
    legend = ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=True, facecolor='white')
    ax.grid(True, which='both', alpha=0.25)

    # Add a) and b) labels
    if j == 0:
        ax.text(0.02, 1.05, f'a) Logistic Zeros Plotted As Upper Limits', transform=ax.transAxes, va='top', ha='left', fontsize=20)
    else:
        ax.text(0.02, 1.05, f'b) Logistic Zeros Plotted As Galaxies Without BHs', transform=ax.transAxes, va='top', ha='left', fontsize=20)


# Plotting and saving
plt.savefig('../../Figures/simulated_data_10x.pdf', format='pdf')
plt.show()


### Simulating 1000 Points Per Posterior Sample

In [52]:
# Getting simulated data. Putting upper limits at zero
hur_flat_100x = copy.deepcopy(hur_arr_100x[sims]).flatten()
ul_idx_flat_100x = np.where(hur_flat_100x == 0)
lin_flat_100x = copy.deepcopy(lin_arr_100x[sims]).flatten()
lin_flat_100x[ul_idx_flat_100x] = 0

bin_edges = np.linspace(0, 14, num=50)  # Adjust num as needed

# Plotting histograms with common bin edges
fig = plt.figure(figsize=(12,13))
plt.hist(data.MBH, bins=bin_edges, alpha=0.5, density=True, color='dodgerblue', label='Observed Data')
plt.hist(lin_flat_100x, bins=bin_edges, alpha=0.5, density=True, color=color_sim, label='Simulated Data')

ax = plt.gca()
ax.set_xlim(-0.5, 13)
ax.set_ylim(0, 0.375)
# Customizing font size for axis labels
# ax.text(5.35, -0.02, r'BH mass $M$', ha='center', va='top', fontsize=25)
# ax.text(7.20, -0.02, r'$\bullet$', ha='center', va='top', fontsize=35)
# ax.text(8.90, -0.02, r'(log $M_\odot$)', ha='center', va='top', fontsize=25)
ax.text(5.70, -0.02, r'BH mass $M$', ha='center', va='top', fontsize=25)
ax.text(7.15, -0.02, r'$\bullet$', ha='center', va='top', fontsize=35)
ax.text(8.60, -0.02, r'(log $M_\odot$)', ha='center', va='top', fontsize=25)
# ax.set_xlabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)
ax.set_ylabel(r'Density', fontsize=25)

# Customizing ticks
ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_major_formatter('{x:.0f}')
ax.xaxis.set_minor_locator(MultipleLocator(0.5))

ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_major_formatter('{x:.1f}')
ax.yaxis.set_minor_locator(MultipleLocator(0.05))

# Remove tick label at beginning of x axis
yticks = ax.yaxis.get_major_ticks()
yticks[1].set_visible(False)

# Adding legend
ax.legend(loc='upper left', fontsize='xx-large', markerscale=2.5, frameon=True, facecolor='white')
ax.grid(True, which='both', alpha=0.25)

plt.savefig('../../Figures/histogram_comparison_100x.pdf', format='pdf')

In [53]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import container
import numpy as np

# Velocity dispersion (sigma) values for plotting
sigma_plot_values = np.linspace(1, 2.8, num=100)

# Plot labels
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8), sharey=False)

for ax, j in zip(axes, range(len(axes))):
    ax.set_xlim(1.0, 2.8)
    ax.set_ylim(-0.5, 11)

    # Plotting simulated data
    for i in sims:
        x_rep = copy.deepcopy(x_rep_arr_100x[i])
        lin = copy.deepcopy(lin_arr_100x[i])
        hur = copy.deepcopy(hur_arr_100x[i])

        upp_lim_idx = np.where(hur == 0)
        non_upp_lim_idx = np.where(hur == 1)
        
        if j == 1:
            lin[upp_lim_idx] = 0
            ax.scatter(x_rep[upp_lim_idx], lin[upp_lim_idx], color=color_sim, marker='v', s=50, alpha=0.3)
        else:
            ax.scatter(x_rep[upp_lim_idx], lin[upp_lim_idx], color=color_sim, marker='v', s=50, edgecolors='black', alpha=0.3)

        ax.scatter(x_rep[non_upp_lim_idx], lin[non_upp_lim_idx], color=color_sim, s=36, alpha=0.3)

    # Plotting true data
    ax.errorbar(stars_upplim.SIG, stars_upplim.MBH, stars_upplim.DMBH, ls='', lw=0.75, color='darkred', marker='.')
    ax.scatter(stars_upplim.SIG, stars_upplim.MBH, color='darkred', marker='v', s=50)
    ax.errorbar(stars_true.SIG, stars_true.MBH, stars_true.DMBH, ls='', lw=0.75, color='darkred', marker='.', label='stars')
    ax.scatter(stars_true.SIG, stars_true.MBH, color='darkred', s=36)

    ax.scatter(stars_upplim_zero.SIG, stars_upplim_zero.MBH, color='darkred', marker='v', s=50)

    ax.errorbar(gas_upplim.SIG, gas_upplim.MBH, gas_upplim.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.')
    ax.scatter(gas_upplim.SIG, gas_upplim.MBH, color='dodgerblue', marker='v', s=50)
    ax.errorbar(gas_true.SIG, gas_true.MBH, gas_true.DMBH, ls='', lw=0.75, color='dodgerblue', marker='.', label='gas')
    ax.scatter(gas_true.SIG, gas_true.MBH, color='dodgerblue', s=36)

    ax.errorbar(reverb_upplim.SIG, reverb_upplim.MBH, reverb_upplim.DMBH, ls='', lw=0.75, color='black', marker='.')
    ax.scatter(reverb_upplim.SIG, reverb_upplim.MBH, color='black', marker='v', s=50)
    ax.errorbar(reverb_true.SIG, reverb_true.MBH, reverb_true.DMBH, ls='', lw=0.75, color='black', marker='.', label='reverberation')
    ax.scatter(reverb_true.SIG, reverb_true.MBH, color='black', s=36)

    ax.errorbar(maser_upplim.SIG, maser_upplim.MBH, maser_upplim.DMBH, ls='', lw=0.75, color='forestgreen', marker='.')
    ax.scatter(maser_upplim.SIG, maser_upplim.MBH, color='forestgreen', marker='v', s=50)
    ax.errorbar(maser_true.SIG, maser_true.MBH, maser_true.DMBH, ls='', lw=0.75, color='forestgreen', marker='.', label='maser')
    ax.scatter(maser_true.SIG, maser_true.MBH, color='forestgreen', s=36)

    ax.scatter(omitted_upplim.SIG, omitted_upplim.MBH, color='grey', marker='v', s=50)
    ax.errorbar(omitted_true.SIG, omitted_true.MBH, ls='', lw=0.75, color='grey', marker='.', label='omitted')
    ax.scatter(omitted_true.SIG, omitted_true.MBH, color='grey', s=36)

    # Customizing font size for axis labels
    ax.text(0.88, 3.85, r'BH mass $M$', ha='center', va='center', rotation=90, fontsize=25)
    ax.text(0.9, 5.7, r'$\bullet$', ha='center', va='center', rotation=90, fontsize=35)
    ax.text(0.88, 7.5, r'(log $M_\odot$)', ha='center', va='center', rotation=90, fontsize=25)
    ax.set_xlabel(r'velocity dispersion $\sigma$ (log km/s)', fontsize=25)
    # ax.set_ylabel(r'BH mass $M_\bullet$ (log $M_\odot$)', fontsize=25)

    # Customizing legend to add upper limit marker and remove error bars
    handles, labels = ax.get_legend_handles_labels()

    new_handles = []

    for h in handles:
        # only needed to edit the errorbar legend entries
        if isinstance(h, container.ErrorbarContainer):
            new_handles.append(h[0])
        else:
            new_handles.append(h)

    ax.tick_params(reset=True)
    ax.tick_params(which='major', direction='in', length=10, top=True, right=True, labelsize=20)
    ax.tick_params(which='minor', direction='in', length=5, top=True, right=True)
    ax.xaxis.set_major_locator(MultipleLocator(0.5))
    ax.xaxis.set_major_formatter('{x:.1f}')
    ax.xaxis.set_minor_locator(MultipleLocator(0.1))

    ax.yaxis.set_major_locator(MultipleLocator(2))
    ax.yaxis.set_major_formatter('{x:.0f}')
    ax.yaxis.set_minor_locator(MultipleLocator(0.5))

    # Remove tick label at beginning of x axis
    xticks = ax.xaxis.get_major_ticks()
    xticks[1].set_visible(False)

    # upper limit marker
    labels.insert(0, 'upper limit')  # modify the index depending on how many plots
    new_handles.insert(0, Line2D([0], [0], color='black', marker='v', ls='', markersize=4))
    labels.insert(6, 'simulated')  # modify the index depending on how many plots
    new_handles.insert(6, Line2D([0], [0], color=color_sim, marker='.', ls=''))

    # ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=False)
    legend = ax.legend(new_handles, labels, markerscale=2.5, loc='upper left', fontsize='large', frameon=True, facecolor='white')
    ax.grid(True, which='both', alpha=0.25)

    # Add a) and b) labels
    if j == 0:
        ax.text(0.02, 1.05, f'a) Logistic Zeros Plotted As Upper Limits', transform=ax.transAxes, va='top', ha='left', fontsize=20)
    else:
        ax.text(0.02, 1.05, f'b) Logistic Zeros Plotted As Galaxies Without BHs', transform=ax.transAxes, va='top', ha='left', fontsize=20)

# Plotting and saving
plt.savefig('../../Figures/simulated_data_100x.pdf', format='pdf')
plt.show()